<h1>GRUPO ZAP: DATA SCIENCE CHALLENGE</h1>

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import zap_helpers as zap

RAW_PATH   = '../data-raw/'
DATA_PATH  = '../data/'

GET_PARSE_SAVE_DATA = False

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)

In [2]:
if GET_PARSE_SAVE_DATA:  
    # GET
    base_url  = 'https://s3.amazonaws.com/grupozap-data-challenge/data/'
    url_train = base_url + 'source-4-ds-train.json.zip'
    url_test  = base_url + 'source-4-ds-test.json.zip'

    zap.download_json(url_train, RAW_PATH, 'train-raw')
    zap.download_json(url_test, RAW_PATH, 'test-raw')
    
    # PARSE
    json_train = RAW_PATH + 'train-raw.json'
    json_test  = RAW_PATH + 'test-raw.json'

    df_train = zap.parse_json(json_train)
    df_test  = zap.parse_json(json_test)
    
    # SAVE
    train_data = DATA_PATH + 'train.pkl'
    test_data  = DATA_PATH + 'test.pkl'
    
    zap.pkl_io(train_data, method='dump', file=df_train)
    zap.pkl_io(test_data, method='dump', file=df_test)

In [3]:
train_data = DATA_PATH + 'train.pkl'
test_data  = DATA_PATH + 'test.pkl'

df_train = zap.pkl_io(train_data)
df_test  = zap.pkl_io(test_data)

# ANÁLISE EXPLORATÓRIA

#### Começaremos explorando uma amostra dos dados e seus tipos, para avaliar se a conversão JSON -> DataFrame está correta.

In [4]:
df_train.head()

,address_city,address_country,address_district,address_geoLocation_location_lat,address_geoLocation_location_lon,address_geoLocation_precision,address_locationId,address_neighborhood,address_state,address_street,address_streetNumber,address_unitNumber,address_zipCode,address_zone,bathrooms,bedrooms,createdAt,description,id,images,listingStatus,owner,parkingSpaces,pricingInfos_businessType,pricingInfos_price,publicationType,publisherId,suites,title,totalAreas,unitTypes,updatedAt,usableAreas,pricingInfos_monthlyCondoFee,pricingInfos_period,pricingInfos_rentalTotalPrice,pricingInfos_yearlyIptu
0,São Paulo,BR,,-23.612923,-46.614222,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Jardim da...,Jardim da Saúde,São Paulo,Rua Juvenal Galeno,53,,04290030,Zona Sul,3,4,2017-02-07T13:21:40Z,"04 dorms sendo 01 suíte e closet, sala de esta...",787c7bd19d,[https://s3-sa-east-1.amazonaws.com/vr.images....,ACTIVE,False,6,SALE,700000,STANDARD,f4603b2b52,1,PRÓXIMO A AVENIDA PRESIDENTE TANCREDO NEVES,388,TWO_STORY_HOUSE,2018-12-06T19:27:12.623Z,388,NaN,NaN,NaN,NaN
1,São Paulo,BR,,-23.643962,-46.593475,RANGE_INTERPOLATED,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Vila Liviero,Vila Santa Teresa (Zona Sul),São Paulo,Rua Juruaba,16,,04187320,Zona Sul,2,3,2016-03-21T18:35:17Z,"03 dorms sendo 01 suíte, sala, sala de jantar,...",4d68c0cdbe,[https://s3-sa-east-1.amazonaws.com/vr.images....,ACTIVE,False,2,SALE,336000,STANDARD,f4603b2b52,1,PRÓXIMO A FACULDADE UNIP CAMPUS ANCHIETA,129,HOME,2018-12-12T13:17:23.547Z,129,NaN,NaN,NaN,NaN
2,São Paulo,,,-23.568559,-46.647452,ROOFTOP,BR>Sao Paulo>NULL>Sao Paulo>Centro>Cerqueira C...,Bela Vista,São Paulo,Avenida Paulista,402,,01311000,,4,0,2018-12-18T23:47:03.425Z,"Andar com 395,70m² de área útil, 04 wcs, 05 va...",e7e0b554ac,[http://static.nidoimovel.com.br/d3d9446802a44...,ACTIVE,False,5,RENTAL,24929,STANDARD,501f6d5e94,0,Excelente Conjunto Comercial na Av. Paulista,None,COMMERCIAL_PROPERTY,NaN,396,4900.0,MONTHLY,29829.0,4040.0
3,São Paulo,,,-23.598023,-46.682781,RANGE_INTERPOLATED,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Vila Olimpia,Vila Olímpia,São Paulo,Rua Alvorada,1190,,04550004,,2,3,2018-10-26T16:18:28.915Z,"Viva melhor e de um jeito novo, com privilégio...",6654d93423,[https://vr-prod-vivapro-images.s3.amazonaws.c...,ACTIVE,False,2,SALE,739643,STANDARD,527c7b0cc0,1,"Apartamento com 80m², 2 vagas próximo ao Parqu...",80,APARTMENT,2018-11-08T16:38:35.434Z,80,686.0,NaN,NaN,1610.0
4,São Paulo,,,-23.582497,-46.653027,RANGE_INTERPOLATED,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Paraiso,Paraíso,São Paulo,Rua Curitiba,380,,04005030,,5,4,2018-12-14T18:06:51.342Z,Sua área é ocupada majoritariamente por instit...,9ffaf676ae,[https://vr-prod-vivapro-images.s3.amazonaws.c...,ACTIVE,False,5,SALE,7520099,STANDARD,527c7b0cc0,4,"Apartamento 332m², 4 suítes à venda próximo ao...",332,APARTMENT,2018-12-14T18:06:55.140Z,3322,6230.0,NaN,NaN,18900.0


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133964 entries, 0 to 133963
Data columns (total 37 columns):
address_city                        133964 non-null object
address_country                     133964 non-null object
address_district                    133964 non-null object
address_geoLocation_location_lat    133953 non-null float64
address_geoLocation_location_lon    133953 non-null float64
address_geoLocation_precision       133953 non-null object
address_locationId                  133964 non-null object
address_neighborhood                133964 non-null object
address_state                       133964 non-null object
address_street                      133964 non-null object
address_streetNumber                133964 non-null object
address_unitNumber                  133964 non-null object
address_zipCode                     133964 non-null object
address_zone                        133964 non-null object
bathrooms                           133051 non-null object
be

#### Removeremos as colunas:
 - *images*, composta por listas com URLs para imagens do imóvel. Não trabalharemos com imagem nesse modelo.
 - *description* e *title*, por serem textuais e necessitarem de processamento de linguagem natural para poderem ser usadas como features.

In [6]:
cols_to_remove = ['images', 'description', 'title']
df_train.drop(axis=1, columns=cols_to_remove, inplace=True)

#### Algumas colunas que deveriam ser numéricas foram lidas como **object**. Isso indica inconsistência nos dados. Um exemplo é a variável **address_unitNumber**. Esperamos que indique o número do apartamento, quando aplicável, mas o preenchimento dos dados é o seguinte:

In [7]:
df_train.address_unitNumber.value_counts()

                                      133933
Apartamento                                4
Edifício Itaipu                            1
Well Pompéia                               1
Campos Eliseos                             1
Sala 22                                    1
21                                         1
Quatro Estações                            1
Ville Bayard                               1
APTO                                       1
Edifício Itaici                            1
191                                        1
41                                         1
CASA                                       1
Condomínio                                 1
Dona Norma                                 1
02                                         1
APARTAMENTO                                1
C                                          1
1140                                       1
APTO 81                                    1
00                                         1
2º Andar  

#### Além da quase totalidade dos dados serem nulos, o preenchimento dos não-nulos é incorreto. Vamos tratar esse e demais casos a seguir.

In [8]:
cols_to_numeric = ['address_streetNumber', 'address_unitNumber', 'bathrooms', 'bedrooms', 'parkingSpaces', 'suites', 'totalAreas', 'usableAreas']

df_train.loc[:, cols_to_numeric] = df_train.loc[:, cols_to_numeric].applymap(zap.convert_to_num)

#### Verificando os dados após a conversão:

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133964 entries, 0 to 133963
Data columns (total 34 columns):
address_city                        133964 non-null object
address_country                     133964 non-null object
address_district                    133964 non-null object
address_geoLocation_location_lat    133953 non-null float64
address_geoLocation_location_lon    133953 non-null float64
address_geoLocation_precision       133953 non-null object
address_locationId                  133964 non-null object
address_neighborhood                133964 non-null object
address_state                       133964 non-null object
address_street                      133964 non-null object
address_streetNumber                89637 non-null float64
address_unitNumber                  7 non-null float64
address_zipCode                     133964 non-null object
address_zone                        133964 non-null object
bathrooms                           133051 non-null float64
bedro

#### Vamos analisar as variáveis categóricas.

In [10]:
distinct_count = df_train.select_dtypes(include=['object']).apply(lambda x: x.value_counts().shape[0])
print(distinct_count)

address_city                          1
address_country                       2
address_district                      3
address_geoLocation_precision         5
address_locationId                 1185
address_neighborhood               1634
address_state                         4
address_street                    12933
address_zipCode                   15925
address_zone                          6
createdAt                        132239
id                               133964
listingStatus                         1
pricingInfos_businessType             2
publicationType                       2
publisherId                        3533
unitTypes                            19
updatedAt                        112004
pricingInfos_period                   3
dtype: int64


#### Algumas possuem poucos valores distintos. Vamos detalhar esses casos.

In [11]:
df_train.loc[:, list(distinct_count[distinct_count <= 6].index)].apply(lambda x: x.value_counts(dropna=False)).T.stack()

address_city                   São Paulo             133964.0
address_country                                      120112.0
                               BR                     13852.0
address_district                                     133951.0
                               Jaraguá                   11.0
                               Jaçanã                     2.0
address_geoLocation_precision  APPROXIMATE             7090.0
                               GEOMETRIC_CENTER       37623.0
                               NO_GEOCODE                30.0
                               RANGE_INTERPOLATED     14137.0
                               ROOFTOP                75073.0
                               NaN                       11.0
address_state                                             3.0
                               São Paulo             133714.0
                               SP                       246.0
                               Santa Catarina             1.0
address_

#### Podemos ver que todas as observações dessa amostra são de imóveis em São Paulo. Consequentemente, podemos excluir as colunas *address_city*, *addres_state* e *address_country*. A variável *listingStatus* pode sair da análise por ser constante. *address_district* será excluída por ser praticamente toda composta por valores nulos.

In [13]:
categorical_to_remove = ['address_city', 'address_state', 'address_country', 'address_district', 'listingStatus']
df_train.drop(axis=1, columns=categorical_to_remove, inplace=True)

cols_to_remove += categorical_to_remove

In [ ]:
id_col = ['id']